# Nettoyage des données des moniteurs

Ce notebook présente un pipeline complet pour le nettoyage des données des moniteurs provenant d’eBay.
Les étapes de nettoyage réalisées sont :
- **Nettoyage de la colonne `Price`** : Extraction et conversion du prix en nombre flottant.
- **Uniformisation de la colonne `Screen Size`** : Nettoyage de la chaîne de caractères et conversion en flottant, avec remplissage des valeurs manquantes par la médiane par marque.
- **Nettoyage de la colonne `Response Time`** : Extraction des valeurs numériques et remplacement des valeurs manquantes par la médiane.
- **Extraction du `Refresh Rate`** : Récupération du taux de rafraîchissement à partir de la colonne correspondante ou du titre.
- **Nettoyage avancé du titre** : Suppression d’informations superflues (termes liés à la taille, au taux de rafraîchissement, à la technologie, etc.) pour ne conserver qu’un titre épuré.
- **Suppression des doublons** : En gardant, en cas de doublon, la ligne ayant le prix le plus bas.

Chaque fonction ou étape est expliquée en détail dans la cellule qui la précède.


In [ ]:
import os
import pandas as pd
import numpy as np
import re

# Dans un notebook, __file__ n'est pas défini ; on utilise le répertoire de travail courant.
base_dir = os.getcwd()

# Définir les chemins relatifs pour les données eBay
raw_data_dir_ebay = os.path.join(base_dir, 'data', 'raw', 'ebay', 'monitors')
cleaned_data_dir_ebay = os.path.join(base_dir, 'data', 'cleaned', 'ebay', 'monitors')

# Créer les dossiers de sortie s'ils n'existent pas
os.makedirs(cleaned_data_dir_ebay, exist_ok=True)

print("Répertoire de données brutes :", raw_data_dir_ebay)
print("Répertoire de données nettoyées :", cleaned_data_dir_ebay)


## Fonction `clean_monitors_ebay`

Cette fonction prend en entrée un DataFrame `df` contenant des informations sur des moniteurs et effectue plusieurs opérations de nettoyage :

1. **Nettoyage de la colonne `Price` :**
   - Si la colonne `Brand` est manquante, on lui attribue la valeur par défaut `"Unknown"`.
   - La fonction interne `clean_price` parcourt chaque valeur de `Price` et extrait uniquement les chiffres (et le point décimal), pour convertir le résultat en un nombre flottant.

2. **Uniformisation de la colonne `Screen Size` :**
   - La fonction interne `clean_screen_size` retire les symboles indésirables (comme `"` ou `inches`) et isole la valeur numérique.
   - Les valeurs manquantes sont ensuite remplacées par la médiane des tailles pour chaque marque, et si nécessaire par la médiane globale.

3. **Nettoyage de la colonne `Response Time` :**
   - La fonction `clean_response_time` extrait la valeur numérique (entier ou décimal) contenue dans la chaîne.
   - Les valeurs manquantes sont remplacées par la médiane de la colonne.

4. **Extraction du `Refresh Rate` :**
   - La fonction `extract_refresh_rate` tente d’extraire le taux de rafraîchissement depuis la colonne `Refresh Rate`.
   - Si cette extraction échoue, elle cherche dans le titre (`Title`) la présence d’un nombre suivi de "Hz".

5. **Nettoyage avancé du titre :**
   - La fonction `clean_title_advanced` supprime divers termes et motifs (par exemple, "Gaming Monitor", les valeurs en Hz ou ms, certaines technologies et formats d’affichage) pour obtenir un titre épuré.

6. **Renommage et suppression de colonnes :**
   - Certaines colonnes sont renommées pour uniformiser la nomenclature (par exemple, `Screen Size` devient `Screen_Size_in`).
   - La colonne `Max_Resolution` est supprimée, puis les lignes avec trop de valeurs manquantes sont éliminées.

7. **Suppression des doublons :**
   - La fonction `remove_duplicates_with_min_price` trie les lignes par `Price` (croissant) et supprime les doublons en gardant la ligne avec le prix minimal parmi celles ayant des caractéristiques identiques.


In [ ]:
def clean_monitors_ebay(df):
    # 1. Nettoyage de la colonne Price
    if 'Brand' not in df.columns:
        print("⚠️ La colonne 'Brand' est absente. Une valeur par défaut sera utilisée.")
        df['Brand'] = 'Unknown'

    def clean_price(price):
        """
        Extrait les chiffres et le point décimal d'une chaîne de caractères et les convertit en float.
        Si la chaîne est vide après extraction, retourne NaN.
        """
        if isinstance(price, str):
            cleaned = ''.join(filter(lambda x: x.isdigit() or x == '.', price))
            return float(cleaned) if cleaned else np.nan
        return price

    df['Price'] = df['Price'].apply(clean_price)

    # 2. Uniformisation de Screen Size
    def clean_screen_size(size):
        """
        Nettoie la valeur de la taille d'écran en retirant les symboles (", inches, etc.) et en extrayant la valeur numérique.
        Retourne la valeur numérique ou NaN si non trouvée.
        """
        if isinstance(size, str):
            size = size.replace('"', '').replace("''", "").replace("inches", "")
            # Récupérer la première chaîne qui représente un nombre
            matches = [s for s in size.split() if s.replace('.', '').isdigit()]
            return float(matches[0]) if matches else np.nan
        return size

    df['Screen Size'] = df['Screen Size'].apply(clean_screen_size)

    # Remplacement des valeurs manquantes de Screen Size
    brand_avg_size = df.groupby('Brand')['Screen Size'].transform('median')
    df['Screen Size'] = df['Screen Size'].fillna(brand_avg_size)
    median_screen_size = df['Screen Size'].dropna().median()
    df['Screen Size'] = df['Screen Size'].fillna(median_screen_size)

    # 3. Nettoyage de la colonne "Response Time"
    def clean_response_time(response_time):
        """
        Extrait la valeur numérique (int ou float) d'une chaîne représentant le temps de réponse.
        Retourne NaN si aucun nombre n'est trouvé.
        """
        if isinstance(response_time, str):
            match = re.search(r'(\d+(\.\d*)?)', response_time)
            return float(match.group(0)) if match else np.nan
        elif isinstance(response_time, (int, float)):
            return float(response_time)
        return np.nan

    df['Response Time'] = df['Response Time'].apply(clean_response_time)
    median_response_time = df['Response Time'].dropna().median()
    df['Response Time'] = df['Response Time'].fillna(median_response_time)

    # 4. Extraction du Refresh Rate
    def extract_refresh_rate(row):
        """
        Extrait le taux de rafraîchissement :
        - D'abord à partir de la colonne 'Refresh Rate' en récupérant le plus grand nombre trouvé.
        - Si non disponible, recherche dans le 'Title' un nombre suivi de 'Hz'.
        Retourne None si aucune valeur n'est trouvée.
        """
        original_value = row['Refresh Rate']
        if pd.notna(original_value):
            matches = re.findall(r'\d+', str(original_value))
            if matches:
                return max(map(int, matches))
        # Recherche dans le titre
        title_matches = re.findall(r'(\d+)\s*Hz', row['Title'], flags=re.IGNORECASE)
        if title_matches:
            return max(map(int, title_matches))
        # Vérification alternative si "HZ" est présent
        if "HZ" in row['Title'].upper():
            m = re.search(r'(\d+)\s*HZ', row['Title'].upper())
            if m:
                return int(m.group(1))
        return None

    df['Refresh Rate'] = df.apply(extract_refresh_rate, axis=1)

    # 5. Renommer les colonnes pour uniformiser le DataFrame
    new_columns = {
        'Screen Size': 'Screen_Size_in',
        'Maximum Resolution': 'Max_Resolution',
        'Aspect Ratio': 'Aspect_Ratio',
        'Refresh Rate': 'Refresh_Rate_Hz',
        'Response Time': 'Response_Time_ms'
    }
    df = df.rename(columns=new_columns)

    # 6. Nettoyage avancé du titre
    def clean_title_advanced(title):
        """
        Nettoie le titre du produit en supprimant :
          - Les mentions de taille (inch, inches, ", etc.)
          - Les résolutions ou formats (FHD, UHD, 4K, etc.)
          - Certains termes techniques et de marketing (Gaming Monitor, HDR, IPS, etc.)
        Le titre est ensuite épuré de caractères spéciaux et d’espaces superflus.
        """
        terms_to_remove = [
            r'\bGaming Monitor\b',
            r'\b\d+\.?\d*\s*Hz\b',
            r'\b\d+\.?\d*\s*ms\b',
            r'\b(HDR\d*|IPS|VA|TN|OLED)\b',
            r'\b(Curved|Flat|UltraWide)\b'
        ]
        # Suppression des mentions de taille
        title = re.sub(r'\b\d+\.?\d*\s*(inch|inches|"|’’|’)\b', '', title, flags=re.IGNORECASE)
        # Suppression des résolutions et formats
        title = re.sub(r'\b(\d{3,4}[x×]\d{3,4}|FHD|HD|UHD|QHD|WQHD|4K)\b', '', title, flags=re.IGNORECASE)
        # Suppression des termes techniques et marketing
        for pattern in terms_to_remove:
            title = re.sub(pattern, '', title, flags=re.IGNORECASE)
        # Nettoyage des caractères spéciaux et des espaces inutiles
        title = re.sub(r'[^\w\s.-]', '', title)
        title = re.sub(r'\s+', ' ', title).strip()
        return title

    df['Title'] = df['Title'].apply(clean_title_advanced)

    # 7. Suppression des doublons et des colonnes superflues
    # On retire la colonne "Max_Resolution" qui n'est plus nécessaire
    if 'Max_Resolution' in df.columns:
        df = df.drop('Max_Resolution', axis=1)
    # Suppression des lignes ayant trop de valeurs manquantes
    df = df.dropna(thresh=df.shape[1] - 2)

    def remove_duplicates_with_min_price(dataframe, columns_to_check, price_column):
        """
        Trie le DataFrame par prix croissant, puis supprime les doublons en gardant la ligne
        avec le prix minimal pour les mêmes caractéristiques.
        """
        dataframe = dataframe.sort_values(by=price_column, ascending=True)
        dataframe = dataframe.drop_duplicates(subset=columns_to_check, keep='first')
        return dataframe

    # Définir les colonnes à utiliser pour détecter les doublons
    columns_to_check_for_duplicates = ['Brand', 'Model', 'Screen_Size_in', 'Refresh_Rate_Hz', 'Response_Time_ms']
    df = remove_duplicates_with_min_price(df, columns_to_check_for_duplicates, 'Price')

    return df


## Traitement des fichiers CSV eBay

Pour chaque fichier CSV présent dans le répertoire des données brutes (`raw_data_dir_ebay`), le script :
1. Lit le fichier dans un DataFrame.
2. Applique la fonction `clean_monitors_ebay` pour nettoyer les données.
3. Sauvegarde le DataFrame nettoyé dans le répertoire de données nettoyées (`cleaned_data_dir_ebay`) en préfixant le nom du fichier par `cleaned_`.


In [ ]:
# Parcourir tous les fichiers CSV dans le répertoire des données brutes eBay
for filename in os.listdir(raw_data_dir_ebay):
    if filename.endswith('.csv'):
        file_path = os.path.join(raw_data_dir_ebay, filename)
        print(f"Traitement du fichier : {file_path}")

        # Lecture du fichier CSV
        df = pd.read_csv(file_path)

        # Application du nettoyage
        cleaned_df = clean_monitors_ebay(df)

        # Définir le nom et le chemin du fichier nettoyé
        cleaned_filename = f"cleaned_{filename}"
        cleaned_file_path = os.path.join(cleaned_data_dir_ebay, cleaned_filename)

        # Sauvegarder le DataFrame nettoyé au format CSV
        cleaned_df.to_csv(cleaned_file_path, index=False)
        print(f"Fichier nettoyé sauvegardé sous : {cleaned_file_path}\n")


# Conclusion

Nous avons ainsi défini un pipeline complet de nettoyage des données pour les moniteurs d’eBay.
Les principales étapes comprenaient :
- La transformation et le nettoyage des colonnes numériques (`Price`, `Screen Size`, `Response Time`, `Refresh Rate`).
- Le nettoyage avancé des titres pour retirer les informations superflues.
- La suppression des doublons en privilégiant les enregistrements avec le meilleur prix.

Ce notebook peut être adapté et étendu en fonction des spécificités des données et des besoins d’analyse.
N’hésitez pas à ajouter des validations ou visualisations supplémentaires pour confirmer la qualité du nettoyage.
